# COVID-19 World (Week 2) - modified SIR

The 2019-2020 COVID-19 pandemic is both a devastating virus and an opportunity for us to join together to explore our understanding of disease, disease transmission, and the factors that we can observe and control to address the spread of disease and its impact.

In this notebook, the primary focus is on the challenges with the available data. Given the high number of anecdotes regarding under-counting, lack of testing, and an overwhelmed medical system, which pieces of the data can we use reliably? If we rely on those, do we end up with reasonable predictions of future data?

## Table of Contents
 - [SIR Model](#section_sir)
 - [Notebook setup](#section_background)
 - [EDA](#section_EDA)
 - [Models](#section_models)
 - [Model Evaluations](#section_model_evaluations)
 - [Conclusion](#section_conclusion)

<a id='section_sir'></a>
## Classic SIR model 
The classic SIR model describes a disease progression for each individual as

**S**usceptible -> **I**nfected -> **R**esolved

and has 3 governing equations
* **dS/dt** = (-1 x BETA x S x I)/N
* **dI/dt** = (BETA x S x I)/N - (GAMMA x I)
* **dR/dt** = GAMMA x I

where **N** is the total population and the two factors 

* **BETA** (transmission factor that moves individuals from S to I)
* **GAMMA** (recovery factor that moves individuals from I to R)

are specific to the disease being modelled.


## Modified SIR model

The model used here is a modified SIR where the modifications are factors to estimate the true counts for I and R, namely 
* I = i_factor * ConfirmedCases
* R = r_factor * Fatalities

### S = the number of susceptible people 
  - Since there has not been a vaccine, this study assumes that the starting value for S is the total population of the country or region.
  - The total population of a country is probably not the best estimator. The number of susceptible people in each densely populated and infected region would likely be a better predictor, and perhaps the density of the population would further improve that. Unfortunately, this level of granularity (cities, towns) is not currently available in the data.
   * In the SIR model, S + I + R = N = total_population at all points in time. We can calculate S after calculating changes to I and R.
  - __Conclusion__: The total population of each region will be used keeping in mind the limitations (assumptions) that this introduces to the model.
  
### I = the number of infected people
 * If we had perfect data, we would have I = ActiveCases.
 * Instead, we have ConfirmedCases which is believed to be under-estimated for a variety of reasons (silent carriers, inability to test all symptomatic individuals, other reporting issues).
 * __Conclusion__: The number of ConfirmedCases will be used to estimate the number of actual Infected cases, using an estimated correction factor, INFECTED = i_factor * ConfirmedCases. 
 
### R = the number of people who are resolved
 * R includes the individuals who are no longer susceptible and no longer actively infected.
 * Resolved = Recovered + Fatalities
 * Cases that were never counted as ConfirmedCases will also not be counted as recoveries.
 * Fatalities are most likely to be accurate as testing is likely to occur when individuals require a diagnostic that provides clarity into the necessary interventions, and many countries require a "Cause of Death" to be recorded on death certificates.
 * __Conclusion__: The number of Fatalities will be used to estimate the number of actual Resolved cases, using an estimated correction factor, RESOLVED = r_factor * Fatalities.
     


## Hypothesis and Model

Essentially, our hypothesis and assumptions are:
 * SIR model is appropriate:
  * No vaccinated or immune individuals.
  * Exposure data is not available so the existence of an "E" state as in SEIR is a hidden node. S->E->I can be approximated by S->I.
  * Quarantines are 'leaky' in that movement and interaction are reduced but not completely halted, and that unconfirmed / silent carriers are the most likely to be moving about, so SEIQR would also contain a hidden node without visibility or data.
 * Number of Fatalities is the most accurate statistic.
 * The amount of under-estimation in historic data is a good predictor of future under-estimation.
 * Factors calculated to correct for under-estimation can be used to predict future reported data.
 * The derivatives (dS/dt, dI/dt, dR/dt where ideally dt->0) can be replaced by delta-S/delta-t etc. with minimal loss of accuracy compared to other error sources:
  * The best available dt at this point is a time delta of 1 day (resolution of the data). 
  * dS, dI, and dR are confined to integers (resolution of counting people).
 
This notebook implements a modified SIR as follows:
  * BETA and GAMMA are estimated based on comparison to the seasonal flu, measles, and estimations made in the public disclosures (news and other reports).
  * i_factor and r_factor are based on EDA of historical data.
  * Prior to calculating (forecasting) the next values, the correction factor is applied to I and R.
  * After calculating the next 'true' step, only the uncorrected values are saved as ConfirmedCases and Fatalities; these form the predictions.
        
Additional data sources:
 1.  Population by country from https://www.kaggle.com/tanuprabhu/population-by-country-2020). 
 1.  Population of 3711 for the Diamond Princess is from https://en.wikipedia.org/wiki/Diamond_Princess_(ship).

*Note: This notebook has room for improvement in several areas; please improve on this if you copy code! For example, some regions did not correctly get population data from the external source and are using the default value instead. i_factor and r_factor EDA could be replaced with an ML search for the best factors.*

<a id='section_background'></a>
## Notebook setup
This section contains imports, globals, and functions that will be used to work with the data.

### Week 1:
 - *Notebook for COVID-19 CA* was developed first. That notebook explored 
  - *Extended data* back to 2020-01-22 (manually added from JHU csse time series)
  - *Initial models*
    - 1) *NAIVE model*, akin to "same as previous day" but adding +5 for ConfirmedCases and +1 for Fatalities. This model performed 
    very poorly (as expected) but was an easy way to get the code set up and the first prediction submitted. 
    I considered this a "pipe cleaner."
    - 2) *SIR modified model*
      - a simple calculation based on per-day-deltas (e.g. dS/dt is the change from one day to the next -- avoided complex math)
      - believing that US testing is still not representative of the full extent of the infected cases (I), and further that 
        R = recoverd + fatalities while only fatalities are reported, the model was modified to SCALE both of these to estimate I and R, as in
        - I = i_factor * ConfirmedCases
        - R = r_factor * Fatalities
 - *Extend to WORLD*. 
   - Kept the modified SIR model with a country-keyed table of parameters. 
   - Ran out of time to customize the parameter table.
 - *Ran out of time* to upload the correctly functioning version, so the results were poor. Fortunately, I learned a lot.

### Week 2:

 - Fit to fatalities. As in the earlier data, I believe that the data for Fatalities is more accurate than the data for ConfirmedCases, 
so the models will be built to be more accurate for the Fatalities at the potential risk of incorrectly 
estimating the ConfirmedCases.
 - SIR parameter prediction. Week 1 was focused (for me) on extending my CA notebook to WORLD data. Week 2 is 
    focused on exploring the parameter predictions and determining which input factors (population total, population density, etc.) 
    provide the best parameter fit. In theory, the parameters that provide the best fit should be CLUES to the 
    true inputs to the system. Note that I refer to these as only clues; population density may really be an indicator of cultural social distancing, housing structure, climate, or some other factor that is the true input.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt

# PEP 484 type hints are easier with this
from typing import Tuple

### Globals

In [ ]:
fname_train = '../input/covid19-global-forecasting-week-2/train.csv'
fname_test = '../input/covid19-global-forecasting-week-2/test.csv'
fname_sub = '../input/covid19-global-forecasting-week-2/submission.csv'

In [ ]:
df_train = pd.read_csv(fname_train,parse_dates=True)
df_test  = pd.read_csv(fname_test, parse_dates=True)
df_sub   = pd.read_csv(fname_sub)

In [ ]:
# Week 2 globals
# Training dates are from 2020-01-22 to 2020-03-25 ... but the end date can extend each day as more data is added
#                                                  ... possibly up to 2020-03-31
# Week 2: Training index goes 29364 and contains 294 locations
START_TRAIN_DATE = df_train['Date'].min()   # Week 2: '2020-01-22'
END_TRAIN_DATE   = df_train['Date'].max()   # Week 2: '2020-03-25'

# Prediction dates are from 3/19/2020 to 4/30/2020 (43 dates per region)
# Prediction index goes to 12642
START_PREDICTION_DATE = df_test['Date'].min() # Week 2: '2020-03-19'
END_PREDICTION_DATE   = df_test['Date'].max() # Week 2: '2020-04-30'

In [ ]:
# Tanu's population by country
fname_pop = '../input/population-by-country-2020/population_by_country_2020.csv'
df_pop = pd.read_csv(fname_pop)

In [ ]:
df_pop.head()


In [ ]:
GLOBAL_FILL_STRING = 'not_provided'

In [ ]:
prediction_scores = {}  # create a dictionary of prediction scores so that I know where to consider improvements

### Functions

In [ ]:
def check_first_ConfirmedCase_exists(df_in:pd.DataFrame,
                                     region_list:set)->int:
    '''Confirm that the data for the region contains at least the first ConfirmedCase
    '''
    count_no_cases = 0
    
    for sname in sname_set:       
        df1 = df_in[df_in['SimpleName'] == sname]   # grab the entries for this location only        
        total_cc = df1['ConfirmedCases'].max()        
        if (total_cc < 1):
            print("{} as 0 ConfirmedCases".format(sname))
            count_no_cases += 1
            
    return count_no_cases

In [ ]:
def evaluate_predictions(df_golden:pd.DataFrame,df_predictions:pd.DataFrame,\
                         num_wh=0, num_extended=0,\
                         model_name="no-name",verbose=False)->Tuple[(float,float)]:
    '''Compare the predicted values in df_predictions to the values in df_golden for where the data was withheld.
        Data in the extended region is ignored.
        
       Note: DataFrames MUST only contain a single prediction series for ONE location.
    '''
    round_places = 4
    
    # changed names df_train  --> df_golden
    #               df_pred   --> df_predictions
    
    last_golden_date = df_golden['DateTime'].max()
    
    last_predicted_date = df_predictions['DateTime'].max()
    first_withheld_date = last_golden_date - (num_wh)*pd.to_timedelta('1D')
    if (verbose):
        print("last_golden_date: {}".format(last_golden_date))
        print("last_predicted_date: {}".format(last_predicted_date))
        print("first_withheld_date: {}".format(first_withheld_date))
    # comparison period spans the first_withheld_date to the last_golden_date inclusive
    
    start_eval_time = first_withheld_date
    end_eval_time   = last_golden_date
     
    df_pred_for_eval = df_predictions[(df_predictions['DateTime'] >= start_eval_time) & 
                                      (df_predictions['DateTime'] <= end_eval_time)]
    df_answers       = df_golden[(df_golden['DateTime'] >= start_eval_time) & 
                                 (df_golden['DateTime'] <= end_eval_time)]
    
    if (verbose):
        print("df_answers is {}".format(df_answers))
        print("df_pred_for_eval is {}".format(df_pred_for_eval))
        
    
    c_rmsle = round(rmsle(list(df_answers['ConfirmedCases']),
                          list(df_pred_for_eval['ConfirmedCases'])),
                    round_places)
    f_rmsle = round(rmsle(list(df_answers['Fatalities']),
                          list(df_pred_for_eval['Fatalities'])),
                    round_places)
    
    if (verbose):
        print("Model {} for {} through {}".format(model_name,start_eval_time, end_eval_time))
        print("   evaluation (ideal --> 0):\n    RMSLE confirmed: {}\n    RMSLE fatalities: {}".format(c_rmsle,f_rmsle))
    
    return c_rmsle, f_rmsle

In [ ]:
def evaluate_predictions2(df_golden:pd.DataFrame,df_predictions:pd.DataFrame,\
                         num_wh=0, num_extended=0,\
                         model_name="no-name",verbose=False)->Tuple[(float,float)]:
    '''Compare the predicted values in df_predictions to the values in df_golden for where the data was withheld.
        Data in the extended region is ignored.
        
       Note: DataFrames MUST only contain a single prediction series for ONE location.
    '''
    round_places = 4
    
    # changed names df_train  --> df_golden
    #               df_pred   --> df_predictions
    
    last_golden_date = df_golden['DateTime'].max()
    
    last_predicted_date = df_predictions['DateTime'].max()
    first_withheld_date = last_golden_date - (num_wh)*pd.to_timedelta('1D')
    if (verbose):
        print("last_golden_date: {}".format(last_golden_date))
        print("last_predicted_date: {}".format(last_predicted_date))
        print("first_withheld_date: {}".format(first_withheld_date))
    # comparison period spans the first_withheld_date to the last_golden_date inclusive
    
    start_eval_time = first_withheld_date
    end_eval_time   = last_golden_date
     
    df_pred_for_eval = df_predictions[(df_predictions['DateTime'] >= start_eval_time) & 
                                      (df_predictions['DateTime'] <= end_eval_time)]
    df_answers       = df_golden[(df_golden['DateTime'] >= start_eval_time) & 
                                 (df_golden['DateTime'] <= end_eval_time)]
    
    if (verbose):
        print("df_answers is {}".format(df_answers))
        print("df_pred_for_eval is {}".format(df_pred_for_eval))
        
    
    c_rmsle = round(rmsle(list(df_answers['ConfirmedCases']),
                          list(df_pred_for_eval['maxConfirmedCases'])),
                    round_places)
    f_rmsle = round(rmsle(list(df_answers['Fatalities']),
                          list(df_pred_for_eval['Fatalities'])),
                    round_places)
    
    if (verbose):
        print("Model {} for {} through {}".format(model_name,start_eval_time, end_eval_time))
        print("   evaluation 2 (ideal --> 0):\n    RMSLE confirmed: {}\n    RMSLE fatalities: {}".format(c_rmsle,f_rmsle))
    
    return c_rmsle, f_rmsle

In [ ]:
# From https://www.kaggle.com/marknagelberg/rmsle-function
import math

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

In [ ]:
# unit test for evaluate_predictions, withheld = 1, extended = 2
golden_data    = {'DateTime':pd.to_datetime(['2020-06-01','2020-06-02','2020-06-03']),
                  'ConfirmedCases':[2,4,8],
                  'Fatalities':[0,0,1],
                  'Province_State':['PS_TBD','PS_TBD','PS_TBD'],
                  'Country_Region':['CR_TBD','CR_TBD','CR_TBD'],
                  'Id':[201,202,203],
                  'SimpleName':['TBD','TBD','TBD']}
predicted_data = {'DateTime':pd.to_datetime(['2020-06-01','2020-06-02','2020-06-03','2020-06-04','2020-06-05']),
                  'ConfirmedCases':[2,4,6,8,10],
                  'Fatalities':[0,0,0,1,2],
                  'SimpleName':['TBD','TBD','TBD','TBD','TBD']}
df_golden = pd.DataFrame.from_dict(golden_data)
df_predicted = pd.DataFrame.from_dict(predicted_data)


unit_c, unit_f = evaluate_predictions(df_golden,df_predicted,1,2,'unit test',False)
assert((unit_c == 0.1777) & (unit_f == 0.4901)),'Failed unit test for evaluate_predictions'

In [ ]:
def create_parameter_database():
    '''Creates a database of parameters for use in the model.
       Note that this uses 
           global parameter sname_set 
           default values   for all parameters (intent to replace later)
    '''
    # Create a database of BETA, GAMMA, i_factor, r_factor for each SimpleName
    #  Initially, just use the same values I used for CA
    #  Next step will be to customize based on the region, e.g. temperature, proximity to international airports, etc.


    # defaults   beta=0.38  gamma=0.14   ifx=30   rfx=1000
    ones_column = np.ones((len(sname_set),))
    beta_column = 0.38 * ones_column
    gamma_column = 0.14 * ones_column
    ifx_column = 30.0 * ones_column
    rfx_column = 1000.0 * ones_column  

    pop_column = 1000000 * ones_column  ## population - default to 1 million

    data = {'beta':beta_column,'gamma':gamma_column,'ifx':ifx_column,'rfx':rfx_column,'population':pop_column}
    return data

In [ ]:
def plot_train_predictions(df_pred_in,df_train_in,title_str,ylim_factor=1.2):
    fig, axes = plt.subplots(3,2,figsize=(12,6))
    plt.tight_layout()    
    
    title_cc = title_str + ' ConfirmedCases'
    axes[0,0].set_title(title_cc)
    df_pred_in.plot(ax=axes[0,0],legend=True,x='DateTime',y='ConfirmedCases',label='p_Confirmed',c='lightgreen')
    df_train_in.plot(ax=axes[0,0],legend=True,x='DateTime',y='ConfirmedCases',label='Confirmed',c='orange')

    title_f = title_str + ' Fatalities'
    axes[0,1].set_title(title_f)
    df_pred_in.plot(ax=axes[0,1],legend=True,x='DateTime',y='Fatalities',label='p_Fatalities',c='green')
    df_train_in.plot(ax=axes[0,1],legend=True,x='DateTime',y='Fatalities',label='Fatalities',c='darkblue')
    
    #Scale y-axis to be the known ConfirmedCases + 20% to ensure model dovetails with known data
    ylim_cc = ylim_factor * df_train_in['ConfirmedCases'].max()
    axes[1,0].set_title(title_cc)
    axes[1,0].set_ylim(0,ylim_cc)
    df_pred_in.plot(ax=axes[1,0],legend=True,x='DateTime',y='ConfirmedCases',label='p_Confirmed',c='lightgreen')
    df_train_in.plot(ax=axes[1,0],legend=True,x='DateTime',y='ConfirmedCases',label='Confirmed',c='orange')
    
    #title_f = title_str + ' Fatalities'
    ylim_f = ylim_factor * df_train_in['Fatalities'].max()
    axes[1,1].set_title(title_f)
    axes[1,1].set_ylim(0,ylim_f)
    df_pred_in.plot(ax=axes[1,1],legend=True,x='DateTime',y='Fatalities',label='p_Fatalities',c='green')
    df_train_in.plot(ax=axes[1,1],legend=True,x='DateTime',y='Fatalities',label='Fatalities',c='darkblue')
    
    try:
        df_pred_in['maxConfirmedCases'].shape
        title_cc = title_str + ' maxConfirmedCases'
        axes[2,0].set_title(title_cc)
        df_pred_in.plot(ax=axes[2,0],legend=True,x='DateTime',y='maxConfirmedCases',label='p_maxConfirmed',c='lightgreen')
        df_train_in.plot(ax=axes[2,0],legend=True,x='DateTime',y='ConfirmedCases',label='Confirmed',c='orange')
    except:
        pass

In [ ]:
try: 
    df_sname['maxConfirmedCases'].shape
    print('yeah')
except:
    print('nah')

<a id='section_EDA'></a>
## EDA

The format and length of the input and output files changes slightly between weeks 1 and 2:

Week 1:
 - Final predictions for 12,212 ForecastId; forecast only ConfirmedCases and Fatalities
 - ForecastID are broken down into
 - 284 forecast locations X 43 dates 3/12/2020 - 4/23/2020 inclusive = 12212 predictions
 
Week 2:

 - Final predictions for 12,642 ForecastId; forecast only ConfirmedCases and Fatalities
 - ForecastID are broken down into
  - 294 forecast locations x 43 dates 3/19/2020 - 4/30/2020 inclusive
 - TRAIN data has been modified from Week 1:
  - Dropped: 
    - latitude
    - longitude
  - Changed:  
    - Province/State -> Province_State
    - Country/Region -> Country_Region

In [ ]:
df_train.head(n=2)

In [ ]:
df_train.tail(n=2)

In [ ]:
df_test.head(n=2)

In [ ]:
df_test.tail(n=2)

In [ ]:
df_sub.tail(n=2)

### Add 2 columns to train.csv and also test.csv:
 - SimpleName = Province_State  __ Country_Region
 - DateTime = Date converted to pandas DateTime format

In [ ]:
df_train[['Province_State','Country_Region']] = df_train[['Province_State','Country_Region']].fillna(GLOBAL_FILL_STRING)
df_test[['Province_State','Country_Region']] = df_test[['Province_State','Country_Region']].fillna(GLOBAL_FILL_STRING)

In [ ]:
df_train['SimpleName'] = df_train['Province_State'] + '__' + df_train['Country_Region']   # for fetching data
df_test['SimpleName'] = df_test['Province_State'] + '__' + df_test['Country_Region']      # for storing predictions

In [ ]:
df_train['DateTime'] = pd.to_datetime(df_train['Date'])   #format = '%Y-%m-%d'
df_test['DateTime']  = pd.to_datetime(df_test['Date'])   #format = '%Y-%m-%d'

In [ ]:
df_test.tail(n=3)

In [ ]:
df_train.tail(n=3)

In [ ]:
df_test.dtypes

### Create a list of the unique locations

In [ ]:
df_train['SimpleName'].nunique()

In [ ]:
sname_set = list(set(df_train['SimpleName']))    # another handy global

In [ ]:
# Make sure there aren't any locations with no ConfirmedCases reported yet
assert(0 == check_first_ConfirmedCase_exists(df_train,sname_set)),"Data contains locations with NO ConfirmedCases!"

### Prepare a table of parameters for each of the unique locations for use in the model

Start with default values for all locations

In [ ]:
# Start with default values for all locations

parameter_data = create_parameter_database()
db_parameters = pd.DataFrame(parameter_data, index=sname_set,\
                            columns=['beta','gamma','ifx','rfx','population'])

In [ ]:
db_parameters.head(n=2)

Modify parameters and customize by location (TODO)

In [ ]:
ctry_col = r'Country (or dependency)'
pop_col  = r'Population (2020)'

In [ ]:
df_pop[df_pop[ctry_col] == 'France'][pop_col].values[0]  # test

In [ ]:
df_pop.head(n=2)

In [ ]:
# TODO: Once the basic functionality generates a valid submission, this is where the creative model tweaks will happen
# UPDATE the db_parameters pd.DataFrame

# populate the parameters with the data from df_pop


for index,row in db_parameters.iterrows():
    state, ctry = index.split('__')   # ctry is ['not_provided', 'Jersey']
    if (state == GLOBAL_FILL_STRING):
        try:
            pop = df_pop[df_pop[ctry_col] == ctry][pop_col].values[0]
        except:
            # 8 or so locations use other names in the population database
            if (ctry == 'Czechia'):
                pop = df_pop[df_pop[ctry_col] == r'Czech Republic (Czechia)'][pop_col].values[0]
            elif (ctry == r'Congo (Kinshasa)'):
                pop = df_pop[df_pop[ctry_col] == r'DR Congo'][pop_col].values[0]
            elif (ctry == r'Congo (Brazzaville)'):
                pop = df_pop[df_pop[ctry_col] == r'Congo'][pop_col].values[0]
            elif (ctry == r'Saint Vincent and the Grenadines'):
                pop = df_pop[df_pop[ctry_col] == r'St. Vincent & Grenadines'][pop_col].values[0]                
            elif (ctry == r'Korea, South'):
                pop = df_pop[df_pop[ctry_col] == r'South Korea'][pop_col].values[0]                                
            elif (ctry == r"Cote d'Ivoire"):
                pop = df_pop[df_pop[ctry_col] == r"Côte d'Ivoire"][pop_col].values[0]   
            elif (ctry == r'Saint Kitts and Nevis'):
                pop = df_pop[df_pop[ctry_col] == r'Saint Kitts & Nevis'][pop_col].values[0]    
            elif (ctry == r'Taiwan*'):
                pop = df_pop[df_pop[ctry_col] == r'Taiwan'][pop_col].values[0]   
            elif (ctry == r'Diamond Princess'):
                pop = 3711   # from Wikipedia  
            else:
                pop=500000   # set a default population of 500k
                print("No population data available for {}, {}".format(state,ctry))
        #print('population is {}'.format(pop))
        #print("{} has population {}".format(ctry,pop))
        db_parameters.loc[index,'population'] = pop
    else:
        # Need to figure out how to handle state populations!
        print("State missing population data: {}, {}".format(state,ctry))

In [ ]:
# Create placeholders for ConfirmedCases and Fatalities
col_len = df_test.shape[0]
zeros_column = np.zeros((col_len,))
df_test['ConfirmedCases'] = zeros_column
df_test['Fatalities'] = zeros_column

# Create a placeholder for max_confirmed -- dataset does not reduce the number of ConfirmedCases to reflect
#   only the active cases
df_test['maxConfirmedCases'] = zeros_column

In [ ]:
df_train_preserve = df_train.copy()  # for debugging and testing
df_test_preserve  = df_test.copy()

In [ ]:
# df_train ready to accept predictions
df_test.tail(n=4)

In [ ]:
# check on France data in df_test -- sanity check of randomly selected location to confirm all 0
df_test[df_test['SimpleName'] == 'not_provided__France'].head(n=22)

<a id='section_models'></a>
## Models

In [ ]:
def prepare_prediction_template(df_in,withhold=0,extend_to_date = END_PREDICTION_DATE,
                                verbose=False):
    '''Prepare a pd.DataFrame that has the known data filled in and the dates ready for the predictions.
        prepare_prediction_template(df_training,withhold=0)   <-- use ALL data, then extend dates
        prepare_prediction_template(df_training,withhold=13)  <-- withhold 13 days, then extend dates
        
       Use withhold=0 for final submission only.
    '''
    # GLOBAL reminders
    # start_train_date = '2020-01-22'
    # end_train_date   = '2020-03-25'
    # start_prediction_date = '2020-03-19'
    # end_prediction_date   = '2020-04-30'
    
    # avoid stupid errors with + or -
    withhold = abs(withhold)
    
    if (withhold != 0):
        # calculate the last date in the training set, then calculate which date to go back to in order
        #    withhold info for training
        # df_train['DateTime'].max() - pd.to_timedelta('13D')     # pattern
        wth_str = str(withhold) + 'D'
        last_date = df_in['DateTime'].max() - pd.to_timedelta(wth_str)
        df_local = df_in[df_in['DateTime']<=last_date].copy()
        if (verbose):
            print("pptemp withhold={} so last_date to copy was {}".format(withhold,last_date))
    else:
        df_local = df_in.copy()
        if (verbose):
            print("pptemp withhold=0 so made complete copy before extension.")
        
    # When extending data, extend these info columns, too
    keep_ps =  df_in.iloc[1]['Province_State']
    keep_cr =  df_in.iloc[1]['Country_Region']
    keep_id =  df_in.iloc[1]['Id'] # not likely to be needed, TBD
    keep_sn =  df_in.iloc[1]['SimpleName']
    
    # Extend the predictions through end_prediction_date; first, find out how far it goes right now
    max_date = df_local['DateTime'].max()

    num_extensions = (pd.to_datetime(extend_to_date) - max_date).days
    if (verbose):
        print("Max date was {} so we need {} extensions.".format(max_date,num_extensions))
    
    # Prepare to extend the dates
    next_date = max_date + pd.to_timedelta('1D')
    #print("types are {} and {}".format(type(next_date),type(end_prediction_date)))
    end_date_datetime = pd.to_datetime(extend_to_date)
    while (next_date <= end_date_datetime):
        #print("Adding a row for new date {}".format(next_date))
        df_local = df_local.append({'Id':0,
                                    'Province_State':keep_ps,
                                    'Country_Region':keep_cr,
                                    'DateTime':next_date,
                                    'ConfirmedCases':0,
                                    'maxConfirmedCases':0,
                                    'Fatalities':0,
                                    'SimpleName':keep_sn},
                                   ignore_index=True)
        next_date = next_date + pd.to_timedelta('1D')
    
    return df_local,num_extensions

In [ ]:
# unit test for prepare_predictions_template
df_golden

In [ ]:
df_1, num_e = prepare_prediction_template(df_golden,withhold=1,extend_to_date='2020-06-09',verbose=True)
print("Output num_e={} and df is \n {}".format(num_e,df_1))

In [ ]:
def calculate_SIR_next_step(S,I,R,N,beta,gamma):
    '''Given the current status of S,I,R, N the total population, and the transition factors, 
        the next step for each category is calculated and returned.
    '''   
    # Governing equations:
    #  dS/dt = (-1*BETA*S*I)/N
    #  dI/dt = (BETA*S*I)/N - GAMMA*I
    #  dR/dt = GAMMA*I
    
    drdt = gamma*I
    dsdt = -1*beta*S*I/N
    didt = -1*dsdt - drdt
    
    # print("dsdt {}  didt {}  drdt {}".format(dsdt,didt,drdt))
    
    S = S + dsdt
    I = I + didt
    R = R + drdt
    
    # Saturate the values
    S = max(S,0)  # max because it's heading to 0
    R = min(R,N)  # min because it's heading to N
    
    if (I<0):
        I = 0
    elif (I>N):
        I = N
    
    return S,I,R  

In [ ]:
def sir_model(df_in,N=100000,beta=.2,gamma=.4,i_factor=1,r_factor=1,verbose=False):
    '''Simple SIR model assuming 
            S = population, N
            I = ConfirmedCases
            R = Fatalities
    '''
    df_local = df_in.copy()   
    first_row = True
    first_CC = False
    
    for index, data_row in df_local.iterrows(): 
        r          = index
        r_previous = index - 1     
        
        if (verbose):
            print("Working on index {} and DateTime {}".format(index,df_local.loc[r,'DateTime']))
            
        c = data_row['ConfirmedCases']
        f = data_row['Fatalities']

        # No changes needed for the first timestamp
        if (first_row):
            first_row = False
            df_local.loc[r,'maxConfirmedCases'] = df_local.loc[r,'ConfirmedCases'] # copy over
        else:
            I = df_local.loc[r_previous,'ConfirmedCases'] * i_factor  # factor to guess at under-estimate
            R = df_local.loc[r_previous]['Fatalities']    * r_factor  # factor to guess at under-estimate
            S = N - (I + R) 
        
            # Start predictions only after the first ConfirmedCase has been found
            if (c > 0):
                first_CC = True   # next time we get c==0, we'll start predicting
                df_local.loc[r,'maxConfirmedCases'] = df_local.loc[r,'ConfirmedCases'] # copy over
            if (first_CC & (c == 0)):    # no prediction for ConfirmedCases
                nS,nI,nR = calculate_SIR_next_step(S,I,R,N,beta,gamma)

                # Resolving the chained indexing issue
                df_local.loc[r,'ConfirmedCases'] = round(nI/i_factor)
                df_local.loc[r,'Fatalities'] = round(nR/r_factor)
                
                # Keep track of the maxConfirmedCases
                df_local.loc[r,'maxConfirmedCases'] = max(df_local.loc[r,'ConfirmedCases'],
                                                          df_local.loc[r_previous,'maxConfirmedCases'])
                if (verbose):
                    print("{} SIR previous {} {} {} is changing to {} {} {}".format(df_local.loc[r,'DateTime'],S,I,R,
                                                    round(nS),round(nI),round(nR)))
            else:       
                if (verbose):
                    print("{} SIR current {} {} {} no prediction needed".format(df_local.loc[r,'DateTime'],S,I,R))
    return df_local

In [ ]:
# Week 2's dates are
# start_prediction_date = '2020-03-19'
# end_prediction_date   = '2020-04-30'

mdates = pd.date_range('2020-03-19', '2020-04-30', freq='D')
date_list = list(mdates.strftime('%Y-%m-%d'))

date_list2 = ['2020-03-19','2020-03-20','2020-03-21','2020-03-22','2020-03-23','2020-03-24','2020-03-25',\
             '2020-03-26','2020-03-27',\
             '2020-03-28','2020-03-29','2020-03-30','2020-03-31','2020-04-01','2020-04-02','2020-04-03','2020-04-04',\
             '2020-04-05','2020-04-06','2020-04-07','2020-04-08','2020-04-09','2020-04-10','2020-04-11','2020-04-12',\
             '2020-04-13','2020-04-14','2020-04-15','2020-04-16','2020-04-17','2020-04-18','2020-04-19','2020-04-20',\
             '2020-04-21','2020-04-22','2020-04-23','2020-04-24','2020-04-24','2020-04-25','2020-04-26','2020-04-27',\
             '2020-04-28','2020-04-29','2020-04-30'
            ]
    
def copy_preditions_to_final_list(location, df_predictions, df_final_list,verbose=False):
    '''Copy the prediction results into the final predictions DataFrame'''
    
    if (verbose):
        print("Copying predictions for {}:".format(location))
        
    assert(100==df_predictions.shape[0]),'Error 1 - Passed in an incorrect predictions DataFrame'    
    df_small = df_predictions[df_predictions['SimpleName'] == location]
    assert(df_small.shape[0]==df_predictions.shape[0]),'Error 2 - Passed in an incorrect predictions DataFrame'  
    # in theory, the length of df_predictions and the length of df_small should be the same  
    
    for date1 in date_list:

        date1_dt = pd.to_datetime(date1)
        
        # create masks to get the correct items lined up
        mask_location = df_final_list['SimpleName'] == location
        mask_date = df_final_list['DateTime'] == date1_dt        
        idx_final = df_final_list[mask_location & mask_date].index[0]
        
        mask_date2 = df_small['DateTime'] == date1_dt
        idx_small = df_small[mask_date2].index[0]        
        
        if (verbose):
            print('idx: final={}  small={}'.format(idx_final, idx_small))
        df_final_list.at[idx_final,'ConfirmedCases'] = df_small.at[idx_small,'ConfirmedCases']
        df_final_list.at[idx_final,'maxConfirmedCases'] = df_small.at[idx_small,'maxConfirmedCases']
        df_final_list.at[idx_final,'Fatalities']     = df_small.at[idx_small,'Fatalities']
        
        #break

In [ ]:
# for each SimpleName, do an SIR forecast with a fixed set of parameters

    # GLOBAL reminders
    # start_train_date = '2020-01-22'
    # end_train_date   = '2020-03-25'
    # start_prediction_date = '2020-03-19'
    # end_prediction_date   = '2020-04-30'
    
def single_model(location, practice_withhold=0, verbose=False):

    this_name = location
    # practice_withhold 
        #  0 means withhold  0 days of data -- use only for final submission!
        # 13 means withhold 13 days of data

    df_sname = df_train[df_train['SimpleName'] == this_name].copy()
    
    df_sname['maxConfirmedCases'] = 0

    df_sname_template,num_extended = prepare_prediction_template(df_sname,(-1*practice_withhold),verbose=verbose)     

    
    df_sname.head()

    # defaults   beta=0.38  gamma=0.14   ifx=30   rfx=1000 N = 500000
    this_beta, this_gamma, this_ifx, this_rfx, this_pop = db_parameters.loc[this_name]

    df_SIR_predictions = sir_model(df_sname_template,N=this_pop,\
                                    beta=this_beta,gamma=this_gamma,\
                                    i_factor = this_ifx, r_factor = this_rfx)

    # Remember that we can ONLY evaluate versus the original training data, so it can only go from start to
    #    end of training data at most.
    if (verbose):
        starter = practice_withhold + num_extended
        print("ends to compare are \n {} \n ***AND*** \n {}".format(df_sname,
                                                                      df_SIR_predictions.iloc[:(-1*num_extended)]))
    rmsle_c, rmsle_f = evaluate_predictions(df_sname,df_SIR_predictions,
                                          practice_withhold,num_extended,
                                          'simple SIR',verbose)
    rmsle_c2, rmsle_f2 = evaluate_predictions2(df_sname,df_SIR_predictions,
                                          practice_withhold,num_extended,
                                          'simple SIR',verbose)
    plot_title = this_name + "  v1 SIR plots"
    #plot_train_predictions(df_SIR_practice,df_sname,plot_title,ylim_factor = 6)

    print("Finished evaluating...{}: {} {}".format(this_name,rmsle_c,rmsle_f))
    print("Finished evaluating 2...{}: {} {}".format(this_name,rmsle_c2,rmsle_f2))
    copy_preditions_to_final_list(this_name, df_SIR_predictions, df_test)
    
    # update dictionary of prediction scores
    prediction_scores[this_name] = [rmsle_c2, rmsle_f2]
    
    return df_SIR_predictions, df_sname, plot_title

In [ ]:
mdf_SIR_practice, mdf_sname, mplot_title = single_model('Hubei__China',practice_withhold=24,verbose=False)

In [ ]:
prediction_scores

##### BEGIN single country investigation ###################

In [ ]:
practice_withhold = 7
verbose = True
this_name = 'Hubei__China'
    # practice_withhold 
    #  0 means withhold  0 days of data -- use only for final submission!
    # 13 means withhold 13 days of data

df_sname = df_train[df_train['SimpleName'] == this_name].copy()
    
df_sname['maxConfirmedCases'] = 0

df_sname_template,num_extended = prepare_prediction_template(df_sname,(-1*practice_withhold),verbose=verbose)     


df_sname.head()

# defaults   beta=0.38  gamma=0.14   ifx=30   rfx=1000 N = 500000
this_beta, this_gamma, this_ifx, this_rfx, this_pop = db_parameters.loc[this_name]

df_SIR_predictions = sir_model(df_sname_template,N=this_pop,\
                                beta=this_beta,gamma=this_gamma,\
                                i_factor = this_ifx, r_factor = this_rfx)

In [ ]:
df_sname

In [ ]:
df_sname_template

In [ ]:
df_SIR_predictions

In [ ]:
    rmsle_c, rmsle_f = evaluate_predictions(df_sname,df_SIR_predictions,
                                          practice_withhold,num_extended,
                                          'simple SIR',verbose)
    rmsle_c2, rmsle_f2 = evaluate_predictions2(df_sname,df_SIR_predictions,
                                          practice_withhold,num_extended,
                                          'simple SIR',verbose)
    plot_title = this_name + "  v1 SIR plots"
    #plot_train_predictions(df_SIR_practice,df_sname,plot_title,ylim_factor = 6)

    print("Finished evaluating...{}: {} {}".format(this_name,rmsle_c,rmsle_f))
    print("Finished evaluating 2...{}: {} {}".format(this_name,rmsle_c2,rmsle_f2))

In [ ]:
##### END single country investigation ###################

In [ ]:
df_sname = df_train[df_train['SimpleName'] == 'not_provided__Germany']

In [ ]:
df_sname_template_1,num_extended = prepare_prediction_template(df_sname,-5)

In [ ]:
num_extended

In [ ]:
# df_sname_template_1.tail(n=40)

In [ ]:
df_SIR_practice = sir_model(df_sname_template_1) # use all defaults

In [ ]:
# df_SIR_practice.tail(n=40)

In [ ]:
# df_sname.tail(n=40)

### *** Single example ***

In [ ]:

# mdf_SIR_practice, mdf_sname, mplot_title = single_model('Alberta__Canada',practice_withhold=24,verbose=True)
# mdf_SIR_practice, mdf_sname, mplot_title = single_model('not_provided__Italy',practice_withhold=24,verbose=True)
mdf_SIR_practice, mdf_sname, mplot_title = single_model('Hubei__China',practice_withhold=24,verbose=False)

In [ ]:
mdf_sname.head()

In [ ]:
plot_train_predictions(mdf_SIR_practice,mdf_sname,mplot_title,ylim_factor = 4)

<a id='section_model_evaluations'></a>
## Model Evaluations

Predictions made by these models will be evaluated by Kaggle using RMSLE metrics. 

By way of doing a sanity check, the existing data and the modelled data should be plotted and compared. 

In [ ]:
selected_country_region = "US"
selected_province_state = "Washington"     # can also use GLOBAL_FILL_STRING

sname = selected_province_state + '__' + selected_country_region
print("Sanity check for {}".format(sname))

In [ ]:
mdf_SIR_practice, mdf_sname, mplot_title = single_model(sname,7)

In [ ]:
plot_train_predictions(mdf_SIR_practice,mdf_sname,mplot_title,ylim_factor = 2)

In [ ]:
db_parameters.loc['Isle of Man__United Kingdom']

In [ ]:
mdf_SIR_practice, mdf_sname, mplot_title = single_model('Isle of Man__United Kingdom',7)
plot_train_predictions(mdf_SIR_practice,mdf_sname,mplot_title,ylim_factor = 2)

In [ ]:
mdf_sname.tail(n=15)

### Finish complete preparation for all locations

In [ ]:
sname_set
mini_set = ['not_provided__Togo',
 'not_provided__Cambodia',
 'Connecticut__US',
 'Jilin__China',
 'Greenland__Denmark',
 'Maine__US',
 'Pennsylvania__US',
 'Quebec__Canada',
 'New Jersey__US',
 'Manitoba__Canada',
 'not_provided__Haiti']

In [ ]:
# full set runs about 3 minutes
# start with clean copies
df_train = df_train_preserve.copy()
df_test = df_test_preserve.copy()

In [ ]:
# Run all models 
for sname in sname_set:
    mdf_SIR_practice, mdf_sname, mplot_title = single_model(sname,0)   # omit last 7 days to check score

In [ ]:
# Sanity check that all models are complete
last_day = df_test[df_test['DateTime'] == pd.to_datetime('2020-04-30')]
print("Final prediction for 2020-04-30 has {} ConfirmedCases and {} Fatalities".format(last_day['ConfirmedCases'].sum(),\
                                                                                          last_day['Fatalities'].sum()))

In [ ]:
last_day

In [ ]:
prediction_scores

In [ ]:
df_scores = pd.DataFrame.from_dict(prediction_scores,orient='index', columns=['cc_score','f_score'])

In [ ]:
df_scores['cc_score'].idxmax()

In [ ]:
df_scores['f_score'].idxmax()

In [ ]:
df_test.shape

<a id='section_conclusion'></a>
## Conclusion

In [ ]:
df_test.head(n=3)

In [ ]:
# write out my results so that I can find issues
# df_test.to_csv('df_test_results2.csv')

In [ ]:
df_sub = df_test.drop(['Province_State','SimpleName','Country_Region','Date','DateTime'],axis=1)

In [ ]:
# final dates 3/19/2020 to 4/30/2020

In [ ]:
df_sub.head()

In [ ]:
df_sub_mod = df_sub.drop('ConfirmedCases', axis=1)

In [ ]:
df_sub_mod.head()

In [ ]:
df_sub_mod.columns = ['ForecastId','Fatalities','ConfirmedCases']
df_sub_mod.head()

In [ ]:
df_sub = df_sub_mod

In [ ]:
df_sub.tail()

In [ ]:
df_sub.to_csv('submission.csv',index=False)

## turned out to have 12642 index lines, which is correct